In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

tensorflow-transform version 1.16.0 is latest stable as of Sep 28, 2025

It is compatible with python 3.9 and 3.10 only.

The current kaggle python docker image uses python 3.11.13.

To use an earlier version of python on Kaggle, one can install conda and create a virtual environment that is based on an earlier version of python. 

activating the venv takes a couple of commands, but succeeds as can be seen by the python version print.

Then, presumably, the notebook can use the virtual environment via python subprocess calls that include the 2 activation statements first, then run the python statements, all in a single bash shell session.

Note that scripts can run in subprocesses in that environment, but this notebook will still be running using the latest python.


In [2]:
!pwd
!echo $HOME

/kaggle/working
/root


%%bash: Executes the entire cell as a shell script. 

In [3]:
%%bash
mkdir -p ~/miniconda3
wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
#install conda and activate to /usr/local
bash ~/miniconda3/miniconda.sh -b -u -p /usr/local
rm ~/miniconda3/miniconda.sh
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

. /usr/local/bin/activate
echo "**$SHELL**"
echo "**$BASH**"
conda init --all

. /root/.bashrc
conda create -q --name my_tfx_env python=3.10 -y
conda activate my_tfx_env
python --version

PREFIX=/usr/local
Unpacking bootstrapper...
Unpacking payload...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /usr/local
accepted Terms of Service for https://repo.anaconda.com/pkgs/main
accepted Terms of Service for https://repo.anaconda.com/pkgs/r
**/bin/bash**
**/usr/bin/bash**
no change     /usr/local/condabin/conda
no change     /usr/local/bin/conda
no change     /usr/local/bin/conda-env
no change     /usr/local/bin/activate
no change     /usr/local/bin/deactivate
no change     /usr/local/etc/profile.d/conda.sh
no change     /usr/local/etc/fish/conf.d/conda.fish
no change   

to activate the conda environment, need to source from conda's activate (which I installed in /usr/local/bin above), then activate the conda virtual environment.

this has to be done for each magic shell cell

In [4]:
%%bash
. /usr/local/bin/activate
conda activate my_tfx_env
python --version

#consider conda install ipykernel
conda install pip

##TODO: install the versions of tensorflow and assoc libraries
pip install pyarrow==10.0.0
pip install apache-beam==2.47.0
pip install tensorflow==2.16.1
pip install tensorflow-transform==1.16.0
pip install tfx==1.16.0
#tf metadata 1.15.0
#tfx-bsl
pip list

Python 3.10.18
2 channel Terms of Service accepted
Channels:
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.

  Using cached pyarrow-10.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.0 kB)
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached pyarrow-10.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (35.2 MB)
Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)

  Using cached apache_beam-2.47.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.0 kB)
  Using cached crcmod-1.7-cp310-cp310-linux_x86_64.whl
  Using cached orjson-3.11.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (41 kB)
  Using cached dill-0.3.1.1-py3-none-any.whl
  Using cached cloudpickle-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached fastavro-1.12.0-cp310-cp310-manylin

The run_command is from
https://www.kaggle.com/code/taylorsamarel/change-python-version-kaggle-v2-taylor-amarel

In [5]:
import subprocess
def run_command(cmd, capture=True, check=False):
    cmds = f". /usr/local/bin/activate; conda activate my_tfx_env; {cmd}"
    try:
        result = subprocess.run(cmds, shell=True, capture_output=capture, text=True, check=check)
        if capture:
            return result.stdout.strip() if result.stdout else result.stderr.strip()
        return result.returncode == 0
    except Exception as e:
        return str(e)

In [6]:
print(run_command("python --version"))

Python 3.10.18


In [7]:
'''
# code to use as a tft test.  put into a script
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam
import apache_beam as beam
import numpy as np

# Define the preprocessing function
def preprocessing_fn(inputs):
  x = inputs['x']
  # Scale the input 'x' to have a mean of 0 and variance of 1
  x_centered = x - tft.mean(x)
  return {
      'x_centered': x_centered
  }

# Define your input data
input_data = [
    {'x': np.array([1.0], dtype=np.float32)},
    {'x': np.array([2.0], dtype=np.float32)},
    {'x': np.array([3.0], dtype=np.float32)},
]

try:
    # Use a pipeline to test the transformation
    with beam.Pipeline() as pipeline:
      with tft_beam.Context(temp_dir='/tmp'):
        dataset_metadata = tft.DatasetMetadata.from_feature_spec({
            'x': tf.io.FixedLenFeature([], tf.float32)
        })
        transformed_dataset, transform_fn = (
            (pipeline | 'CreateData' >> beam.Create(input_data))
            .with_input_types(dataset_metadata.schema)
            | 'AnalyzeAndTransform' >> tft_beam.AnalyzeAndTransformDataset(
                preprocessing_fn
            )
        )
        (
            transformed_dataset
            | 'PrintOutput' >> beam.Map(print)
        )
except Exception as ex:
  print(f'error: {ex}')
'''

"\n# code to use as a tft test.  put into a script\nimport tensorflow as tf\nimport tensorflow_transform as tft\nimport tensorflow_transform.beam as tft_beam\nimport apache_beam as beam\nimport numpy as np\n\n# Define the preprocessing function\ndef preprocessing_fn(inputs):\n  x = inputs['x']\n  # Scale the input 'x' to have a mean of 0 and variance of 1\n  x_centered = x - tft.mean(x)\n  return {\n      'x_centered': x_centered\n  }\n\n# Define your input data\ninput_data = [\n    {'x': np.array([1.0], dtype=np.float32)},\n    {'x': np.array([2.0], dtype=np.float32)},\n    {'x': np.array([3.0], dtype=np.float32)},\n]\n\ntry:\n    # Use a pipeline to test the transformation\n    with beam.Pipeline() as pipeline:\n      with tft_beam.Context(temp_dir='/tmp'):\n        dataset_metadata = tft.DatasetMetadata.from_feature_spec({\n            'x': tf.io.FixedLenFeature([], tf.float32)\n        })\n        transformed_dataset, transform_fn = (\n            (pipeline | 'CreateData' >> beam.C